# Organize the Data

In [4]:
import numpy as np

In [5]:
#Create references to important directories we will use over and over
import os, sys
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = LESSON_HOME_DIR+'/data/'

## Action Plan
1. Create Validation and Sample sets
2. Rearrange image files into their respective directories 
3. Finetune and Train model
4. Generate predictions
5. Validate predictions
6. Submit predictions to Kaggle

## Create validation set and sample

In [12]:
#Create directories
%mkdir results
%cd $DATA_HOME_DIR
%mkdir valid

C:\Users\Flo\Source\Repos\Cervical_Cancer_Comp\data


In [13]:
#creating respecting class dirs in valid
%cd valid/
%ls
%mkdir Type_1
%mkdir Type_2
%mkdir Type_3

C:\Users\Flo\Source\Repos\Cervical_Cancer_Comp\data\valid
 Volume in Laufwerk C: hat keine Bezeichnung.
 Volumeseriennummer: 661E-8808

 Verzeichnis von C:\Users\Flo\Source\Repos\Cervical_Cancer_Comp\data\valid

17.05.2017  11:07    <DIR>          .
17.05.2017  11:07    <DIR>          ..
               0 Datei(en),              0 Bytes
               2 Verzeichnis(se), 67.787.304.960 Bytes frei


In [14]:
# current dir
%pwd

u'C:\\Users\\Flo\\Source\\Repos\\Cervical_Cancer_Comp\\data\\valid'

In [15]:
%cd $DATA_HOME_DIR/train

C:\Users\Flo\Source\Repos\Cervical_Cancer_Comp\data\train


In [16]:
from glob import glob

In [17]:
%ls
g = glob('**/*.jpg')
shuf = np.random.permutation(g)
validation_dir = DATA_HOME_DIR+'valid/' 

for i in range(444): os.rename(shuf[i], validation_dir + shuf[i])

 Volume in Laufwerk C: hat keine Bezeichnung.
 Volumeseriennummer: 661E-8808

 Verzeichnis von C:\Users\Flo\Source\Repos\Cervical_Cancer_Comp\data\train

17.05.2017  11:07    <DIR>          .
17.05.2017  11:07    <DIR>          ..
09.03.2017  00:06             6.148 .DS_Store
17.05.2017  11:07    <DIR>          results
09.03.2017  00:37    <DIR>          Type_1
09.03.2017  00:37    <DIR>          Type_2
09.03.2017  00:37    <DIR>          Type_3
               1 Datei(en),          6.148 Bytes
               6 Verzeichnis(se), 67.787.300.864 Bytes frei


In [18]:
# check valid dir for type 1
%cd $DATA_HOME_DIR
%ls
%cd valid/Type_1
%ls -1 | wc -l

C:\Users\Flo\Source\Repos\Cervical_Cancer_Comp\data
 Volume in Laufwerk C: hat keine Bezeichnung.
 Volumeseriennummer: 661E-8808

 Verzeichnis von C:\Users\Flo\Source\Repos\Cervical_Cancer_Comp\data

17.05.2017  11:07    <DIR>          .
17.05.2017  11:07    <DIR>          ..
09.03.2017  00:37    <DIR>          test
16.05.2017  20:14     2.071.977.419 test.7z
17.05.2017  11:07    <DIR>          train
16.05.2017  21:37     5.944.334.666 train.7z
17.05.2017  11:07    <DIR>          valid
               2 Datei(en),  8.016.312.085 Bytes
               5 Verzeichnis(se), 67.785.396.224 Bytes frei
C:\Users\Flo\Source\Repos\Cervical_Cancer_Comp\data\valid\Type_1


Der Befehl "wc" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [17]:
# check valid dir for type 2
%cd ..
%cd Type_2
% ls -1 | wc -l

/home/paperspace/NBs/Cervical_Cancer_Comp/data/valid
/home/paperspace/NBs/Cervical_Cancer_Comp/data/valid/Type_2
229


In [18]:
# check valid dir for type 2
%cd ..
%cd Type_3
% ls -1 | wc -l

/home/paperspace/NBs/Cervical_Cancer_Comp/data/valid
/home/paperspace/NBs/Cervical_Cancer_Comp/data/valid/Type_3
139


In [19]:
# Create single 'unknown' class for test set
%cd $DATA_HOME_DIR
%cd test
%mkdir unknown
%mv *.jpg unknown/

C:\Users\Flo\Source\Repos\Cervical_Cancer_Comp\data
C:\Users\Flo\Source\Repos\Cervical_Cancer_Comp\data\test


ERROR:root:Line magic function `%mv` not found.


# Creating the Model

In [1]:
# imports
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications import imagenet_utils
from keras.applications.xception import preprocess_input
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras.utils.np_utils import to_categorical
from keras import backend as k

import csv

Using TensorFlow backend.


In [6]:
train_data_dir = DATA_HOME_DIR+'/train'
test_data_dir = DATA_HOME_DIR+'/test'
validation_data_dir = DATA_HOME_DIR+'/valid'


nb_train_samples = 1037
nb_validation_samples = 444

if we are using Inception or Xception, we need to set the inputShape  to 299×299 pixels, followed by updating preprocess  to use a separate pre-processing function that performs a different type of scaling.

In [29]:
# hyper parameters for model
nb_classes = 3  # number of classes
img_width, img_height = 299, 299  # change based on the shape/structure of your images
batch_size = 16  # try 4, 8, 16, 32, 64, 128, 256 dependent on CPU/GPU memory capacity (powers of 2 values).
nb_epoch = 100  # number of iteration the algorithm gets trained. 1 As test run

momentum = 0.9  # sgd momentum to avoid local minimum
transformation_ratio = .05  # how aggressive will be the data augmentation/transformation

### Loading Pre-trained Model using imagenet dataset weight

In [8]:
# Pre-Trained CNN Model using imagenet dataset for pre-trained weights
base_model = Xception(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)
print('Model Loaded!')

Model Loaded!


## Defining own top model block

In [9]:
# Top Model Block
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(nb_classes, activation='sigmoid')(x)
#model = Sequential()
#model.add(Flatten())
#model.add(Dense(256, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(nb_classes, activation='sigmoid'))

In [10]:
# add your top layer block to your base model
model = Model(base_model.input, predictions)

In [11]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 149, 149, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, 149, 149, 32)  128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, 149, 149, 32)  0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [12]:
i = 0
for layer in model.layers:
    i+=1

print(i)

136


In [15]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all layers of the based model that is already pre-trained.
for layer in model.layers[:127]:
    layer.trainable = False

In [16]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 149, 149, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, 149, 149, 32)  128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, 149, 149, 32)  0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [17]:
# Read Data and Augment it: Make sure to select augmentations that are appropriate to your images.
# To save augmentations un-comment save lines and add to your flow parameters.
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   shear_range=0.2,
                                   zoom_range=0.2,)

In [18]:
validation_datagen = ImageDataGenerator(rescale=1. / 255)

In [19]:
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                        target_size=(img_width, img_height),
                                                        batch_size=batch_size,
                                                        class_mode='categorical')

Found 1037 images belonging to 3 classes.


In [20]:
validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                                  target_size=(img_width, img_height),
                                                                  batch_size=batch_size,
                                                                  class_mode='categorical')

Found 444 images belonging to 3 classes.


In [21]:
model.compile(optimizer='nadam',
                  loss='categorical_crossentropy',  # categorical_crossentropy if multi-class classifier
                  metrics=['accuracy'])

In [22]:
# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [23]:
# See: https://github.com/fchollet/keras/issues/5475
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
print("\nStarting to Tune Model\n")
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=nb_epoch,
    validation_data=validation_generator,
validation_steps=nb_validation_samples // batch_size,
callbacks=callbacks_list)
print('Finished tuning')


Starting to Tune Model

Epoch 1/100
64/64 [==============================] - 221s - loss: 0.8438 - acc: 0.5440 - val_loss: 1.1404 - val_acc: 0.5602
Epoch 2/100
64/64 [==============================] - 212s - loss: 0.8365 - acc: 0.5794 - val_loss: 1.2699 - val_acc: 0.5000
Epoch 3/100
64/64 [==============================] - 231s - loss: 0.8016 - acc: 0.5635 - val_loss: 1.2755 - val_acc: 0.5509
Epoch 4/100
64/64 [==============================] - 227s - loss: 0.7767 - acc: 0.5845 - val_loss: 1.2590 - val_acc: 0.5833
Epoch 5/100
64/64 [==============================] - 216s - loss: 0.7264 - acc: 0.6122 - val_loss: 1.2750 - val_acc: 0.5532
Epoch 6/100
64/64 [==============================] - 216s - loss: 0.7329 - acc: 0.6430 - val_loss: 1.3127 - val_acc: 0.4769
Epoch 7/100
64/64 [==============================] - 212s - loss: 0.6893 - acc: 0.6536 - val_loss: 1.3762 - val_acc: 0.5116
Epoch 8/100
64/64 [==============================] - 204s - loss: 0.6521 - acc: 0.6748 - val_loss: 1.1816 -

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model_xception.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_xception.h5")
print("Saved model to disk")

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# TODOs:

- more aggresive data augmentation
- more aggressive dropout
- use of L1 and L2 regularization (also known as "weight decay")
- fine-tuning one more convolutional block (alongside greater regularization)


# Predicting on test set

In [36]:
# load json and create model
json_file = open("model_xception.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("weights-improvement-11-0.60.hdf5")

In [37]:
# Read Data
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(img_width, img_height),
                                                  batch_size=batch_size,
                                                  shuffle=False)

Found 512 images belonging to 1 classes.


In [48]:
# Calculate class posteriors probabilities
y_probabilities = model.predict_generator(test_generator, 32)
print(len(y_probabilities))

512


In [49]:
# Calculate class labels
y_classes = to_categorical(y_probabilities)
filenames = [filename.split('\\')[1] for filename in test_generator.filenames]
ids = [filename.split('.')[0] for filename in filenames]

In [50]:
results_path = LESSON_HOME_DIR+'/results/'
results_name = 'predictions.csv'
# save results as a csv file in the specified results directory
with open(os.path.join(results_path, results_name), 'w') as file:
    writer = csv.writer(file)
    writer.writerow(('image_name', 'Type_1', 'Type_2', 'Type_3'))
    writer.writerows(zip(filenames, y_probabilities[:, 0], y_probabilities[:, 1], y_probabilities[:, 2]))

In [51]:
print(len(filenames))

512


256
